In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2025-02-26 14:52:16--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
connected. to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... 
200 OKequest sent, awaiting response... 
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-02-26 14:52:16 (8.66 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import minsearch

In [4]:
import json

In [7]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [15]:
documents[2]


{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [18]:
q = 'the course has already started, can I still enroll?'
index.fit(documents)

In [20]:
index.text_matrices

{'question': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 8938 stored elements and shape (948, 2051)>,
 'text': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 47683 stored elements and shape (948, 6711)>,
 'section': <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 3651 stored elements and shape (948, 86)>}

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [29]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [39]:
import google.generativeai as genai

def llm(prompt, model_name="gemini-1.5-pro"): # or "gemini-pro"
    """
    Generates a response from the Gemini API.

    Args:
        prompt: The user's input prompt.
        model_name: The name of the Gemini model to use.

    Returns:
        The generated text response.
    """
    genai.configure(api_key=os.getenv("GEMINI_API_KEY")) # Replace with your API key

    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)

    return response.text

In [40]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [41]:
rag(query)

'For Java Kafka, navigate to the project directory and run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`\n\nFor Python Kafka, create and activate a virtual environment:\n\n```bash\npython -m venv env\nsource env/bin/activate  #(or env/Scripts/activate on Windows)\npip install -r ../requirements.txt\n```\nThen, run your Python Kafka files within the activated environment.  Remember to run `source env/bin/activate` each time you start a new terminal session.\n'